In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df_train = pd.read_csv(r'train.csv')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [3]:
from sklearn.preprocessing import MinMaxScaler
mm_scaler = MinMaxScaler()


x_input = df_train.drop(columns=['label']).values
x = mm_scaler.fit_transform(x_input).reshape(-1, 28, 28)
y = pd.get_dummies(df_train['label'])

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [16]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers import BatchNormalization

from keras.layers.convolutional import Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(28, 28, 1), activation='tanh'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='tanh'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation='tanh'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
model.fit(x_train, y_train, batch_size=32, epochs=15, validation_data=(x_test, y_test))

Epoch 1/15
1050/1050 [==============================] - 42s 39ms/step - loss: 0.2064 - accuracy: 0.9392 - val_loss: 0.0849 - val_accuracy: 0.9751
Epoch 2/15
1050/1050 [==============================] - 47s 45ms/step - loss: 0.0676 - accuracy: 0.9798 - val_loss: 0.0622 - val_accuracy: 0.9813
Epoch 3/15
1050/1050 [==============================] - 52s 50ms/step - loss: 0.0531 - accuracy: 0.9833 - val_loss: 0.0576 - val_accuracy: 0.9831
Epoch 4/15
1050/1050 [==============================] - 44s 42ms/step - loss: 0.0389 - accuracy: 0.9884 - val_loss: 0.0493 - val_accuracy: 0.9843
Epoch 5/15
1050/1050 [==============================] - 48s 46ms/step - loss: 0.0346 - accuracy: 0.9896 - val_loss: 0.0452 - val_accuracy: 0.9862
Epoch 6/15
1050/1050 [==============================] - 43s 41ms/step - loss: 0.0302 - accuracy: 0.9908 - val_loss: 0.0466 - val_accuracy: 0.9852
Epoch 7/15
1050/1050 [==============================] - 45s 43ms/step - loss: 0.0291 - accuracy: 0.9904 - val_loss: 0.0453 -

In [18]:
df_test = pd.read_csv('test.csv')

x_out = df_test.values
x_output = mm_scaler.fit_transform(x_out).reshape(-1, 28, 28)
y_pred = np.argmax(model.predict(x_output), axis=1).reshape(-1, 1)

875/875 [==============================] - 7s 8ms/step


In [19]:
df = pd.DataFrame(y_pred, columns=['Label'])
df.index += 1
df = df.rename_axis('ImageId')
print(df.head)

<bound method NDFrame.head of          Label
ImageId       
1            2
2            0
3            9
4            0
5            3
...        ...
27996        9
27997        7
27998        3
27999        9
28000        2

[28000 rows x 1 columns]>


In [20]:
df.to_csv('submission.csv')